In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
from scipy.misc import imsave
import scipy.ndimage as nd
import matplotlib.pyplot as plt
from google.colab import drive
import h5py
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import time
from keras import backend as K
import scipy.ndimage as nd
from keras.models import Sequential,load_model
from keras import regularizers
from keras import initializers
from keras import metrics
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Convolution2D, MaxPooling2D, BatchNormalization, ReLU, LeakyReLU 
from keras import layers
from keras import models
import tensorflow as tf

In [0]:
drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# download TinyImageNet
! git clone https://github.com/seshuad/IMagenet

In [0]:
def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open('IMagenet/tiny-imagenet-200/wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict
  
def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open('IMagenet/tiny-imagenet-200/words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])
        
    return result

def get_data(id_dict):

    print('starting loading data')
    train_data, val_data, test_data = [], [], []
    train_labels, val_labels, test_labels = [], [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [nd.imread('IMagenet/tiny-imagenet-200/train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(450)]
        
        train_labels_ = np.array([[0]*200]*450)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()
        
        val_data += [nd.imread('IMagenet/tiny-imagenet-200/train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(450, 500)]
        
        val_labels_ = np.array([[0]*200]*50)
        val_labels_[:, value] = 1
        val_labels += val_labels_.tolist()

    for line in open('IMagenet/tiny-imagenet-200/val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(nd.imread('IMagenet/tiny-imagenet-200/val/images/{}'.format(img_name), mode='RGB'))

        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))

    return np.array(train_data), np.array(train_labels), np.array(val_data), np.array(val_labels), np.array(test_data), np.array(test_labels)
  
def make_augmented_train(id_dict):
    if not(os.path.isdir('drive/My Drive/tiny-imagenet-200-A')):
        print('making dir')
        os.mkdir('drive/My Drive/tiny-imagenet-200-A')
        os.mkdir('drive/My Drive/tiny-imagenet-200-A/train')
        os.mkdir('drive/My Drive/tiny-imagenet-200-A/test')
  
    crops = [((4, 60), (4, 60)),
             ((0, 56), (0, 56)),
             ((8, 64), (8, 64)),
             ((0, 56), (8, 64)),
             ((8, 64), (0, 56))]
    
    trans = [((4, 60), (4, 60)),
             ((8, 64), (8, 64)),
             ((0, 56), (0, 56)),
             ((8, 64), (0, 56)),
             ((0, 56), (8, 64))]
    
    print('starting creating augmeneted data')
    train_data, train_labels = [], []
    t = time.time()
    
    progress = 0
    for key, value in id_dict.items():
        print(progress)
        progress += 1
        if not(os.path.isdir('drive/My Drive/tiny-imagenet-200-A/train/{}'.format(key))):
            os.mkdir('drive/My Drive/tiny-imagenet-200-A/train/{}'.format(key))
            os.mkdir('drive/My Drive/tiny-imagenet-200-A/train/{}/images'.format(key))
        count = 0
        for crp, trs in zip(crops, trans):
            crop_x, crop_y = crp
            tran_x, tran_y = trs
            for mirror in [-1, 1]:
                for i in range(450):
                    img = nd.imread('IMagenet/tiny-imagenet-200/train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB')[:, ::mirror][crop_x[0]:crop_x[1], crop_y[0]:crop_y[1]]
                    processed_img = np.zeros(shape = (64, 64, 3))
                    processed_img[tran_x[0]:tran_x[1], tran_y[0]:tran_y[1]] = img
                    
                    imsave('drive/My Drive/tiny-imagenet-200-A/train/{}/images/{}_{}.JPEG'.format(key, key, str(count)), processed_img)
                    count += 1

    print('finished creating augmeneted data, in {} seconds'.format(time.time() - t))
  
def shuffle_data(train_data, train_labels, val_data, val_labels):
    size = len(train_data)
    train_idx = np.arange(size)
    np.random.shuffle(train_idx)
    
    size = len(val_data)
    val_idx = np.arange(size)
    np.random.shuffle(val_idx)

    return train_data[train_idx], train_labels[train_idx], val_data[val_idx], val_labels[val_idx]
  
def write_results(model, comments, train_history, filename):
    file = open('drive/My Drive/results/' + filename, 'w')
    acc, loss, val_acc, val_loss = train_history['acc'], train_history['loss'], train_history['val_acc'], train_history['val_loss']
    train_top5, valid_top5 = train_history['top_k_categorical_accuracy'], train_history['val_top_k_categorical_accuracy']
    
    model.summary(print_fn=lambda x: file.write(x + '\n'))
    file.write(comments + '\n')
    
    for metric in acc, loss, train_top5, val_acc, val_loss, valid_top5:
        file.write('*')
        for value in metric:
            file.write(str(value) + ' ')
        file.write('\n')
        
    file.close()
    
def Dense_reg(width, activation, l2, init):
    if l2 == 0: return Dense(width, activation=activation, kernel_initializer=init)
    else: return Dense(width, activation=activation, kernel_regularizer=regularizers.l2(l2), kernel_initializer=init)
    

def cnn_1(dropout = 0.0, l2 = 0):
    model = Sequential ()
    init = initializers.glorot_normal(seed=0)

    model.add(Convolution2D(64, (5, 5), padding='same', activation='relu', strides=(2,2), input_shape=(64, 64, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Convolution2D(128, (5, 5), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Convolution2D(256, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    

    model.add(Flatten())
    model.add(Dense_reg(256, 'relu', l2, init))
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense_reg(256, 'relu', l2, init))
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense(200, activation='softmax'))

    return model
  
  
def cnn_2(dropout = 0.0, l2 = 0.0):
    model = Sequential()

    model.add(Convolution2D(64, (5, 5), padding='same', strides=(2,2), input_shape=(64, 64, 3)))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Convolution2D(128, (5, 5), padding='same'))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Convolution2D(128, (5, 5), padding='same'))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Convolution2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    

    model.add(Flatten())
    model.add(Dense_reg(256, None, l2))
    model.add(LeakyReLU())
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense_reg(256, None, l2))
    model.add(LeakyReLU())
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense(200, activation='softmax'))

    return model
  
def cnn_3(dropout = 0.0, l2 = 0.0):
    model = Sequential()
    init = initializers.glorot_normal(seed=0)

    model.add(Convolution2D(64, (5, 5), padding='same', strides=(2,2), input_shape=(64, 64, 3), kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(Convolution2D(64, (5, 5), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Convolution2D(128, (5, 5), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Convolution2D(128, (5, 5), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Convolution2D(256, (3, 3), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Convolution2D(256, (3, 3), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    

    model.add(Flatten())
    model.add(Dense_reg(512, None, l2, init))
    model.add(LeakyReLU())
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense_reg(512, None, l2, init))
    model.add(LeakyReLU())
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense(200, activation='softmax', kernel_initializer=init))

    return model
  
def cnn_4(dropout = 0.0, l2 = 0.0):
    model = Sequential()
    init = initializers.glorot_normal(seed=0)

    model.add(Convolution2D(64, (5, 5), padding='same', strides=(2,2), input_shape=(64, 64, 3), kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(Convolution2D(64, (5, 5), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Convolution2D(128, (5, 5), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Convolution2D(128, (5, 5), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Convolution2D(256, (3, 3), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Convolution2D(256, (3, 3), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    

    model.add(Flatten())
    model.add(Dense_reg(1024, None, l2, init))
    model.add(LeakyReLU())
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense_reg(1024, None, l2, init))
    model.add(LeakyReLU())
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense(200, activation='softmax'))

    return model
    
def cnn_5(dropout = 0.0, l2 = 0.0):
    model = Sequential()
    init = initializers.glorot_normal(seed=0)
    
    model.add(Convolution2D(64, (5, 5), padding='same', strides=(2,2), input_shape=(64, 64, 3), kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(Convolution2D(64, (5, 5), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Convolution2D(256, (5, 5), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Convolution2D(256, (5, 5), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Convolution2D(512, (3, 3), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Convolution2D(512, (3, 3), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Convolution2D(512, (3, 3), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Convolution2D(512, (3, 3), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Convolution2D(512, (3, 3), padding='same', kernel_initializer=init))
    model.add(LeakyReLU())
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense_reg(1024, None, l2, init))
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense_reg(1024, None, l2, init))
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense(200, activation='softmax'))

    return model
    
def resnet_baseline(x, dropout):
    
    def residual_block(y, nb_channels, kernel_size):

        shortcut = y

        y = layers.Conv2D(nb_channels, kernel_size=kernel_size, strides=(1, 1), padding='same')(y)
        y = LeakyReLU()(y)
        y = layers.BatchNormalization()(y)

        y = layers.Conv2D(nb_channels, kernel_size=kernel_size, strides=(1, 1), padding='same')(y)
        
        y = layers.add([shortcut, y])
        y = LeakyReLU()(y)
        y = layers.BatchNormalization()(y)

        return y
    
    x = layers.Conv2D(48, kernel_size=(7, 7), strides=(2, 2), padding='same')(x)
    x = LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(96, kernel_size=(3, 3), padding='same')(x)
    x = LeakyReLU()(x)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 96, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 96, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 96, (3, 3))
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Conv2D(192, kernel_size=(3, 3), padding='same')(x)
    x = LeakyReLU()(x)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 192, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 192, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 192, (3, 3))
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(2048, activation = 'relu')(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(2048, activation = 'relu')(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(200, activation = 'softmax')(x)
    
    return x
  
  
def resnet_stanford(x, dropout):
  
    
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = residual_block(x, 64, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 64, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 64, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 64, (3, 3))
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 128, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 128, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 128, (3, 3))
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 256, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 256, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 256, (3, 3))
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Conv2D(512, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 512, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 512, (3, 3))
    x = layers.BatchNormalization()(x)
    x = residual_block(x, 512, (3, 3))
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation = 'relu')(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(512, activation = 'relu')(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(200, activation = 'softmax')(x)
    
    return x
  
  
def residual_block(y, nb_channels, kernel_size, init):

    shortcut = y

    y = layers.Conv2D(nb_channels, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_initializer=init)(y)
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)

    y = layers.Conv2D(nb_channels, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_initializer=init)(y)
    y = layers.BatchNormalization()(y)

    y = layers.add([shortcut, y])
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)

    return y
  
  
def resnet_1(x):
    init = initializers.glorot_normal(seed=0)
    
    x = layers.Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(64, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 64, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Conv2D(128, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 128, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Conv2D(256, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 256, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(512, kernel_initializer=init)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(512, kernel_initializer=init)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(200, activation = 'softmax')(x)
    
    return x
  
  
def resnet_2(x):
    init = initializers.glorot_normal(seed=0)
    
    x = layers.Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(64, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 64, (5, 5), init)
    x = residual_block(x, 64, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Conv2D(128, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 128, (5, 5), init)
    x = residual_block(x, 128, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Conv2D(256, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 256, (5, 5), init)
    x = residual_block(x, 256, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Conv2D(512, kernel_size=(3, 3), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 512, (3, 3), init)
    x = residual_block(x, 512, (3, 3), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(512, kernel_initializer=init)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(512, kernel_initializer=init)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(200, activation = 'softmax')(x)
    
    return x
  
  
def resnet_3(x):
    init = initializers.glorot_normal(seed=0)
    
    x = layers.Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(64, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 64, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Conv2D(128, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 128, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(256, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 256, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(1024, kernel_initializer=init)(x)
    x = layers.Dropout(0.25)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(1024, kernel_initializer=init)(x)
    x = layers.Dropout(0.25)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(200, activation = 'softmax')(x)
    
    return x
  
def resnet_4(x):
    init = initializers.glorot_normal(seed=0)
    
    x = layers.Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(64, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 64, (5, 5), init)
    x = residual_block(x, 64, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Conv2D(128, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 128, (5, 5), init)
    x = residual_block(x, 128, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    
    x = layers.Conv2D(256, kernel_size=(5, 5), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 256, (5, 5), init)
    x = residual_block(x, 256, (5, 5), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2D(512, kernel_size=(3, 3), padding='same', kernel_initializer=init)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = residual_block(x, 512, (3, 3), init)
    x = residual_block(x, 512, (3, 3), init)
    x = layers.MaxPool2D(pool_size=(2, 2), padding='same')(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(1024, kernel_initializer=init)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Dense(1024, kernel_initializer=init)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Dense(200, activation = 'softmax')(x)
    
    return x
    
  
def cnn_1_leaky(dropout = 0.0, l2 = 0):
    model = Sequential ()
    init = initializers.glorot_normal(seed=0)

    model.add(Convolution2D(64, (5, 5), padding='same', strides=(2,2), input_shape=(64, 64, 3)))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Convolution2D(128, (5, 5), padding='same'))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Convolution2D(256, (3, 3), padding='same'))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    

    model.add(Flatten())
    model.add(Dense_reg(256, None, l2, init))
    model.add(LeakyReLU())
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense_reg(256, None, l2, init))
    model.add(LeakyReLU())
    if dropout > 0: model.add(Dropout(dropout))
    model.add(Dense(200, activation='softmax'))

    return model

In [0]:
train_data, train_labels, val_data, val_labels, test_data, test_labels = get_data(get_id_dictionary())
train_data, train_labels, val_data, val_labels = shuffle_data(train_data, train_labels, val_data, val_labels)

In [0]:
# example run
model = cnn_1()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

train_history = model.fit(train_data, train_labels, batch_size=1000, epochs=30, validation_data=(val_data, val_labels))

write_results(model,  "no regularisation, no augmentation", train_history, 'cnn_1.txt')
del model